In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gurobipy as gp
from gurobipy import GRB
import tracemalloc
import networkx as nx

In [2]:
tracemalloc.start()

In [3]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 500
DATA_NAME = "auto0"
TOPO_NAME = "4"

task = pd.read_csv("../../data/utilization/utilization_5_5.csv")
network = pd.read_csv("../../data/utilization/utilization_topology.csv")

# task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
# network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [4]:
'''
This method assume that traffics are all the same period.
'''

'\nThis method assume that traffics are all the same period.\n'

## 1. Model

In [5]:
M = int(1e16)

In [6]:
m = gp.Model("RTNS2016-nowait")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01


Network model

In [7]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['dproc'] = np.ceil(row['t_proc'])
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

Task model

In [8]:
task_var = {}

In [9]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [10]:
task_attr = {}
graph = nx.from_numpy_array(net)
## Assume task is strictly periodic
for i, row in task.iterrows():
    task.loc[i,'route'] = str(nx.shortest_path(graph, row['src'], eval(row['dst'])[0]))
    task_var.setdefault(i, {})
    route = eval(task.loc[i, 'route'])
    task_attr.setdefault(i, {})
    task_attr[i]['route'] = route
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})
        task_var[i][link]['dtrans'] = np.ceil(row['size'] * 8)
        if _i == 0:
            ## This one must not cantains processing delay
            task_var[i][link]['D'] = task_var[i][link]['dtrans']
        else:
            task_var[i][link]['D'] = task_var[i][str((route[_i - 1], a))]['D'] \
            + net_var[eval(link)[0]]['dproc'] + task_var[i][link]['dtrans']

In [11]:
t = m.addMVar(shape=(len(task)), vtype=GRB.INTEGER , name="release")
for i in range(len(task)):
    route = eval(task.loc[i, 'route'])
    first_link = str((route[0], route[1]))
    m.addConstr(0 <= t[i])
    m.addConstr(t[i] <= task.loc[i]['period'] - task_var[i][first_link]['dtrans'])

## 2. Constraints

\begin{aligned}
\min C_{m a x} & \\
\text { subject to } & \\
\forall\left\{O_{i, k}, O_{j, l}\right\} \in K: & \\
t_{j}-t_{i}-D_{i, k}+D_{j, l-1}+d^{\text {prop }}+d^{\text {proc }} & \leq c x_{i, k, j, l} \\
\forall\left\{O_{i, k}, O_{j, l}\right\} \in K: & \\
t_{i}-t_{j}-D_{j, l}+D_{i, k-1}+d^{p r o p}+d^{\text {proc }} & \leq c\left(1-x_{i, k, j, l}\right)
\end{aligned}

In [13]:
'''
Processing delay shouldn't be counted in the frame overlap constraint, 
since all we need to do is make sure there isn't a collision between window openning statuses.
'''

"\nProcessing delay shouldn't be counted in the frame overlap constraint, \nsince all we need to do is make sure there isn't a collision between window openning statuses.\n"

In [14]:
## i < j REQUIRED TO BE MERGED INTO GITREPO
for i, j in tqdm([(i,j) for i in task_var for j in task_var if i < j]):
    i_r, j_r = task_attr[i]['route'], task_attr[j]['route']
    i_t, j_t = task.loc[i].period, task.loc[j].period
    lcm = np.lcm(i_t, j_t)
    for k, l in [(k, l) for k in range(len(i_r) - 1) for l in range(len(j_r) - 1)]:
        if str((i_r[k], i_r[k+1])) == str((j_r[l], j_r[l+1])):
            link = str((i_r[k], i_r[k+1]))
            for a, b in [
                            (a, b) for a in range(0, int(lcm / task.loc[i].period))
                                for b in range(0, int(lcm / task.loc[j].period))
                        ]:
                temp = m.addVar(vtype=GRB.BINARY,
                                    name="%d%d%d%d%s" % (i, j, a, b, link))
                m.addConstr(
                    (t[j] + b * j_t) - (t[i] + a * i_t) - task_var[i][link]['D'] + task_var[i][link]['dtrans']
                    + task_var[j][link]['D'] <= M * temp
                )
                m.addConstr(
                    (t[i] + a * i_t) - (t[j] + b * j_t) - task_var[j][link]['D'] + task_var[j][link]['dtrans']
                    + task_var[i][link]['D'] <= M * (1 - temp)
                )

100%|██████████| 55/55 [00:00<00:00, 1029.62it/s]


In [15]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 104 rows, 52 columns and 268 nonzeros
Model fingerprint: 0xd78e65c1
Variable types: 0 continuous, 52 integer (41 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+16]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 1e+16]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 22 rows and 0 columns
Presolve time: 0.00s
Presolved: 82 rows, 52 columns, 246 nonzeros
Variable types: 0 continuous, 52 integer (41 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolera

## Output Schedule

In [176]:
GCL = []
for i in task_var:
    for e in task_var[i]:
        start = t[i].x + task_var[i][e]['D'] - task_var[i][e]['dtrans']
        end = start + task_var[i][e]['dtrans']
        queue = 0
        tt = task.loc[i, 'period']
        for k in range(int(LCM / tt)):
            GCL.append(
                [e, queue, (start + k * tt) * macrotick, (end + k * tt) * macrotick, LCM * macrotick]
            )


In [177]:
## Offset
OFFSET = []
for i in task_var:
    offset = t[i].x
    OFFSET.append(
        [i, 0, (task.loc[i,'period'] - offset) * macrotick]
    )    

In [178]:
ROUTE = []
for i, row in task.iterrows():
    route = task_attr[i]['route']
    for h, v in enumerate(route[:-1]):
        ROUTE.append(
            [i, (v, route[h + 1])]
        )

In [179]:
QUEUE = []
for i in task_var:
    for e in task_var[i]:
        QUEUE.append([i, 0, eval(e), 0])

In [180]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTNS2016_nowait-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTNS2016_nowait-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTNS2016_nowait-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTNS2016_nowait-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

In [181]:
m.Runtime

0.8641791343688965

In [182]:
print(tracemalloc.get_traced_memory()[1] / 1024 / 1024)


6.733712196350098


In [183]:
tracemalloc.stop()

: 